<a href="https://colab.research.google.com/github/isaacfinberg/Novozymes-Enzyme-Stability-Prediction/blob/develop/Rosetta_10_30_Louis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!pip install pandas
!pip install sentence_transformers
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  
!mkdir -p ~/.kaggle                                          
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log 
!chmod 600 ~/.kaggle/kaggle.json                            
!kaggle config set -n competition -v novozymes-enzyme-stability-prediction   
!kaggle competitions download -c novozymes-enzyme-stability-prediction --force
!unzip -o *.zip >> log

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
- competition is now set to: novozymes-enzyme-stability-prediction
  0% 0.00/7.06M [00:00<?, ?B/s]
100% 7.06M/7.06M [00:00<00:00, 79.8MB/s]


In [9]:
SEED = 43
import os
import random as rn
import numpy as np
import tensorflow as tf

os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)

truncate_size = 256

In [10]:
!pip install pyrosettacolabsetup

import os
import pandas as pd
# levinthal / paradox
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
import pyrosetta
pyrosetta.init()
from pyrosetta.teaching import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
PyRosetta-4 2022 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2022.41+release.28dc2a1757fe7c8bf9b661dd8d6cd903c305e581 2022-10-14T12:31:19] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r333 2022.41+release.28dc2a1757f 28dc2a1757fe7c8bf9b661dd8d6cd903c305e581 http://www.pyrosetta.org 2022-10-14T12:31:19
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.7/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1577771101 seed_offset=0 real_seed=1577771101
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1577771101 RG_type=mt19937
PyRosetta-4 2022 [

In [35]:
relaxed_path = '/content/gdrive/MyDrive/Kaggle/Rosetta/wildtypeA/'
files = os.listdir(relaxed_path)
relaxed_pdbs = []
for file in files:
    if file.endswith(".pdb"):
        relaxed_pdbs.append(file)
mutation_to_pdb = {}
mutation_to_pdb['mutation'] = []
mutation_to_pdb['path'] = []
for file in relaxed_pdbs:
    mutation_to_pdb['mutation'].append(file.split('_')[1])
    mutation_to_pdb['path'].append(file)
mutation_to_pdb_df = pd.DataFrame(mutation_to_pdb)
print(mutation_to_pdb_df.head())


  mutation                         path
0    A101T  wildtypeA_A101T_relaxed.pdb
1    A101S  wildtypeA_A101S_relaxed.pdb
2    A101N  wildtypeA_A101N_relaxed.pdb
3    A101K  wildtypeA_A101K_relaxed.pdb
4    A101I  wildtypeA_A101I_relaxed.pdb


In [36]:
scores = []
sfxn = pyrosetta.rosetta.core.scoring.get_score_function()
for i in range(len(mutation_to_pdb_df)):
    pose = pyrosetta.pose_from_pdb(os.path.join(relaxed_path, mutation_to_pdb_df.iloc[i]['path']))
    scores.append(sfxn(pose))
    
mutation_to_pdb_df['scores'] = scores
mutation_to_pdb_df.head()

mutation_to_pdb_df.to_csv(relaxed_path + 'NOVO_wt_relaxed_scores.csv')

Streaming output truncated to the last 5000 lines.
core.conformation.Conformation: current variant for 177 CYD
core.conformation.Conformation: Found disulfide between residues 52 188
core.conformation.Conformation: current variant for 52 CYS
core.conformation.Conformation: current variant for 188 CYS
core.conformation.Conformation: current variant for 52 CYD
core.conformation.Conformation: current variant for 188 CYD
core.import_pose.import_pose: File '/content/gdrive/MyDrive/Kaggle/Rosetta/wildtypeA/wildtypeA_S131L_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 47 177
core.conformation.Conformation: current variant for 47 CYS
core.conformation.Conformation: current variant for 177 CYS
core.conformation.Conformation: current variant for 47 CYD
core.conformation.Conformation: current variant for 177 CYD
core.conformation.Conformation: Found disulfide between residues 52 188
core.conformation.Conformation: current 

In [38]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata

scores = pd.read_csv(relaxed_path + 'NOVO_wt_relaxed_scores.csv')
scores.head()

def find_mut(row):
    mut = row.mutant_seq
    seq = row.sequence
    same = True
    for i,(x,y) in enumerate(zip(seq,mut)):
        if x!=y: 
            same = False
            break
    row['WT'] = seq[i]
    row['position'] = i+1
    if not same:
        if len(mut) < len(seq):
            row['MUT'] = 'X'
        else:
            row['MUT'] = mut[i]
    else: 
        row['position'] = -1
        row['MUT'] = 'X'
    row['mutation_key'] = row['WT']+str(row['position']) + row['MUT']
    return row

In [39]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  
!mkdir -p ~/.kaggle                                          
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log 
!chmod 600 ~/.kaggle/kaggle.json                            
!kaggle config set -n competition -v novozymes-enzyme-stability-prediction   
!kaggle competitions download -c novozymes-enzyme-stability-prediction --force 
!unzip -o *.zip >> log

- competition is now set to: novozymes-enzyme-stability-prediction
  0% 0.00/7.06M [00:00<?, ?B/s]
100% 7.06M/7.06M [00:00<00:00, 151MB/s]


In [40]:
novo_test = pd.read_csv("test.csv")
novo_test

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [41]:
scores

,Unnamed: 0,mutation,path,scores
0,0,A101T,wildtypeA_A101T_relaxed.pdb,-621.702353
1,1,A101S,wildtypeA_A101S_relaxed.pdb,-621.561406
2,2,A101N,wildtypeA_A101N_relaxed.pdb,-621.750326
3,3,A101K,wildtypeA_A101K_relaxed.pdb,-621.886211
4,4,A101I,wildtypeA_A101I_relaxed.pdb,-619.524579
...,...,...,...,...
2331,2331,Y218F,wildtypeA_Y218F_relaxed.pdb,-619.932278
2332,2332,Y218D,wildtypeA_Y218D_relaxed.pdb,-620.498818
2333,2333,Y218C,wildtypeA_Y218C_relaxed.pdb,-615.526494
2334,2334,Y218A,wildtypeA_Y218A_relaxed.pdb,-615.928811


In [42]:
novo_test = novo_test.rename({'protein_sequence': 'mutant_seq', 'seq_id': 'source_df_id'}, axis = 1)
novo_test['sequence'] = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
novo_test = novo_test.apply(find_mut,axis=1)
novo_test = novo_test.join(scores.set_index('mutation'), on='mutation_key')
novo_test['scores'] = -novo_test['scores']
novo_test.loc[novo_test['scores'].isna(), 'scores'] = novo_test.loc[~novo_test['scores'].isna()].quantile(q=0.25)['scores']
novo_test['scores_rank'] = rankdata(novo_test['scores'])
novo_test.head()

,source_df_id,mutant_seq,pH,data_source,sequence,WT,position,MUT,mutation_key,Unnamed: 0,path,scores,scores_rank
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,L,17,E,L17E,1162.0,wildtypeA_L17E_relaxed.pdb,621.184323,2267.0
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,L,17,K,L17K,1161.0,wildtypeA_L17K_relaxed.pdb,621.254853,2271.0
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,L,17,X,L17X,NaN,NaN,613.820542,623.5
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,K,18,C,K18C,915.0,wildtypeA_K18C_relaxed.pdb,617.874844,1519.0
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,K,18,F,K18F,914.0,wildtypeA_K18F_relaxed.pdb,610.671530,353.0


In [47]:
submission_rosetta_scores = novo_test[['source_df_id','scores_rank']]
submission_rosetta_scores = submission_rosetta_scores.rename({'source_df_id': 'seq_id', 'scores_rank': 'tm'}, axis = 1)
submission_rosetta_scores.to_csv('submission_rosetta_scores', index=False)

preds = submission_rosetta_scores.tm.values

In [48]:
pY = pd.DataFrame(preds, index=range(31390,len(preds)+31390), columns=['tm'])
ToCSV = lambda df_tmp, fname: df_tmp.round(5).to_csv(f'{fname}.csv', index_label='seq_id')
ToCSV(pY, '/content/gdrive/MyDrive/Kaggle/Novozymes_Submission')